# 🚚 Smart Logistics: IoT Data Simulation

This notebook simulates IoT-generated data for a smart logistics tracking system. 

It mimics shipment tracking by generating random sensor data such as temperature, humidity, GPS location, timestamps, and delivery status. 

This helps simulate how logistics companies like FedEx or Amazon use real-time data from IoT devices to monitor their shipments.

## 📦 Objectives
🌎 Simulate real-time shipment tracking data using Python.

🌎 Understand how IoT devices gather, timestamp, and report critical data.

🌎 Generate datasets for analysis and visualization later.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

We’ll store our simulated data in a subfolder called `data/`.

In [7]:
os.makedirs("../data", exist_ok=True)

Below, we generate 100 records of shipment tracking data. Each record includes:
- Timestamps: current time, pickup time, estimated arrival
- Shipment ID
- GPS coordinates
- Environmental conditions (temperature, humidity)
- Shipment status
- Alert flags if values exceed acceptable thresholds

In [ ]:
num_records = 100  # Number of records to generate
data = []

for _ in range(num_records):
    # random timestamp within the last 24 hours
    timestamp = datetime.now() - timedelta(minutes=np.random.randint(0, 1440))
    
    # pickup datetime  between 1 and 6 hours before the shipment timestamp
    pickup_datetime = timestamp - timedelta(hours=np.random.randint(1, 6))
    
    # estimated arrival datetime between 1 and 2 days (24 to 48 hours) after the shipment timestamp
    estimated_arrival = timestamp + timedelta(hours=np.random.randint(24, 49))  # 49 to include 48 hours

    # random temperature between 2°C and 8°C for the shipment (e.g., cold storage)
    temp = round(np.random.uniform(2.0, 8.0), 2)
    
    # random humidity value between 50% and 80%
    humidity = round(np.random.uniform(50, 80), 2)
# also ADD THE ff: asset id, Logistics_Delay, Logistics_Delay_Reason
    record = {
        "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),  # Converted to a string format
        "pickup_datetime": pickup_datetime.strftime("%Y-%m-%d %H:%M:%S"),  # Converted to string format
        "shipment_id": f"SHIP{np.random.randint(1000, 9999)}",  # random shipment ID (e.g., SHIP1234)
        "location_lat": round(np.random.uniform(14.0, 14.8), 6),  # Random latitude (e.g., somewhere in the Philippines)
        "location_long": round(np.random.uniform(120.5, 121.5), 6),  # Random longitude
        "temperature_c": temp,  # Temperature of the shipment in Celsius
        "humidity_percent": humidity,  # Humidity level for the shipment
        "status": np.random.choice(["In Transit", "Delivered", "Delayed", "Out for Delivery"]),  # Random status for the shipment
        "estimated_arrival": estimated_arrival.strftime("%Y-%m-%d %H:%M:%S"),  # Converted to string format
        "temperature_alert": temp < 2.5 or temp > 7.5,  # an alert if the temperature is out of the acceptable range (2.5°C to 7.5°C)
        "humidity_alert": humidity < 55 or humidity > 75  # an alert if humidity is outside of the 55%-75% range
    }
    
    # Add the generated record to the list of data
    data.append(record)

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert datetime columns to string format for JSON export
df['timestamp'] = df['timestamp'].apply(str)
df['pickup_datetime'] = df['pickup_datetime'].apply(str)
df['estimated_arrival'] = df['estimated_arrival'].apply(str)

We’ll export the dataset in two formats:
- CSV for spreadsheet tools
- JSON for API and web usage

In [ ]:
# Save to CSV and JSON
df.to_csv("../data/logistics_data.csv", index=False)

import json

with open("../data/logistics_data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

In [6]:
df.head()

,timestamp,pickup_datetime,shipment_id,location_lat,location_long,temperature_c,humidity_percent,status,estimated_arrival,temperature_alert,humidity_alert
0,2025-05-06 21:45:39,2025-05-06 20:45:39,SHIP7761,14.322510,120.514200,5.94,68.98,Delayed,2025-05-08 17:45:39,False,False
1,2025-05-06 07:52:39,2025-05-06 02:52:39,SHIP2438,14.286105,121.153909,6.61,74.63,In Transit,2025-05-07 16:52:39,False,False
2,2025-05-06 10:18:39,2025-05-06 09:18:39,SHIP1221,14.777829,121.478578,5.30,64.91,In Transit,2025-05-07 23:18:39,False,False
3,2025-05-06 03:23:39,2025-05-06 01:23:39,SHIP7530,14.722706,121.126457,6.51,65.38,Delivered,2025-05-07 19:23:39,False,False
4,2025-05-06 18:25:39,2025-05-06 14:25:39,SHIP7234,14.190084,120.853943,5.52,71.86,Delivered,2025-05-07 23:25:39,False,False
